In [ ]:
#model parameters
import numpy as np
import samples as ls
import math
import model_types


# networks used for thesis:
# RNNs:
# 0/1/2_9
# 10/11_34, 12_36 (34 used sgd, 36 adam, not really a difference)
# DNNs:
# 11_39 (sigmoid)
# 11_45 (relu)

# quick overview of the modes:
# 1. load_test: Load the test data for final evaluation. It is only used for testing and not training
# 2. predict_mode: No training. Load all data uncut/without unrolling. 
#                  Makes recurrent networks stateful (they remember their state between runs). This enables the live demo.
# 3. None of the above: Training mode, used to train the networks.


#--------- TRAINING MODES --------------
#load final test data for final evaluation
load_test = False

#enable to make RNNs stateful (save state between batches) and to not unroll training data for whole sample comparison
predict_mode = False

#load the network from saved file (needs to be False if network is not yet created)
load_rnn = True if not predict_mode else False
#only load the weights, not the architecture
load_only_weights = False if not load_rnn else False
#model to use (num=architecture, type=labels/task)
model_num = 34
model_type = 11

train_network = False if not load_test else False
live_demo = False


#--------- MODEL DATA --------------
model_id = '%s_%s' % (model_type, model_num)
model_path = 'tmp/keras/nn-%s.hdf5' % model_id
weights_path = 'tmp/keras/nn-%s_weights.hdf5' % model_id
labels,sample_ids,filename = model_types.get_labels(model_type, load_test)


#--------- MODEL HYPERPARAMETERS --------------
accel,gyro,compass = 'xyz','xyz',''
overlap_step = 1
use_labels_data = False #-> if true, output at each timestep is compared (should be true for first two tasks)
use_lstm = True
use_sgd = False #-> if false, adamoptimizer is used
use_rnn = True #use a recurrent cell
input_steps = 50 #both set to 50 as the default setting for the angle network
unroll_steps = 50
activation = 'sigmoid'
load_samples = True
dropout_prob = 0 #for recurrent cells
dropout_dense = False #for dense layers
complete_unroll = False
add_absolutes = False
add_moving_average = False
normalize = False


#--------- MODEL DEFINITIONS --------------
if model_num == 0:
    input_steps = None
    unroll_steps = 100
    overlap_step = 50
    use_labels_data = True
    use_sgd = True
    hidden_layers = [50]
elif model_num == 4:
    input_steps = None
    unroll_steps = 100
    overlap_step = 50
    use_labels_data = True
    use_sgd = True
    hidden_layers = [50]
elif model_num == 6:
    input_steps = None
    unroll_steps = 100
    overlap_step = 50
    use_labels_data = True
    hidden_layers = [10]
elif model_num == 7:#trained with new data
    input_steps = None
    unroll_steps = 100
    overlap_step = 50
    use_labels_data = True
    hidden_layers = [50]
elif model_num == 8:
    input_steps = None
    unroll_steps = 100
    overlap_step = 50
    use_labels_data = True
    complete_unroll = True
    add_absolutes = True
    add_moving_average = True
    hidden_layers = [18*2]
elif model_num == 9:
    input_steps = None
    unroll_steps = 100
    overlap_step = 50
    use_labels_data = True
    complete_unroll = True
    hidden_layers = [14]
elif model_num == 10:#trained with normalized data
    input_steps = None
    unroll_steps = 100
    overlap_step = 50
    use_labels_data = True
    complete_unroll = True
    normalize = True
    hidden_layers = [14]
    
    
elif model_num ==15:
    input_steps = None
    unroll_steps = 100
    overlap_step = 50
    use_labels_data = True
    hidden_layers = []
elif model_num ==16:
    input_steps = None
    unroll_steps = 100
    overlap_step = 50
    use_labels_data = True
    hidden_layers = [14]
    
    
    

elif model_num == 33:
    use_rnn = True
    use_lstm = True
    use_labels_data = False
    load_samples = False
    use_sgd = True
    add_absolutes = True
    add_moving_average = True
    hidden_layers = [24]

elif model_num == 32:
    use_rnn = True
    use_lstm = True
    use_labels_data = False
    load_samples = False
    use_sgd = True
    add_absolutes = True
    add_moving_average = True
    hidden_layers = [18*2]

elif model_num == 34:
    use_rnn = True
    use_lstm = True
    use_labels_data = False
    load_samples = False
    use_sgd = True
    hidden_layers = [24]
elif model_num == 36:
    use_rnn = True
    use_lstm = True
    use_labels_data = False
    load_samples = False
    hidden_layers = [24]
    
    
elif model_num == 40:
    use_rnn = False
    activation = 'sigmoid'
    hidden_layers = [50,50,50]
elif model_num == 41:
    use_rnn = False
    activation = 'sigmoid'
    hidden_layers = [50,24]
elif model_num == 39:
    use_rnn = False
    activation = 'sigmoid'
    hidden_layers = [50,24]
elif model_num == 42:
    use_rnn = False
    hidden_layers = [24,24,12]
elif model_num == 45:
    use_rnn = False
    activation = 'relu'
    hidden_layers = [24,24,24,24,12]
elif model_num == 46:
    use_rnn = False
    activation = 'relu'
    hidden_layers = [24,24,24,24,24,12]
elif model_num == 47:
    use_rnn = False
    activation = 'relu'
    dropout_dense = True
    hidden_layers = [24,24,24,24,24,12]
elif model_num == 48:
    use_rnn = False
    activation = 'relu'
    dropout_dense = True
    hidden_layers = [48,24,24,24,24,24]
else:
    raise ValueError('Unknown model')

    
input_dim = len(accel)+len(gyro)+len(compass)

classes = len(labels)

#predict mode/test mode -> dont unroll for task 1 and 2
if predict_mode:
    input_steps = None
    unroll_steps = None
    overlap_step = 1
#however, for angle task always unroll
if model_type >= 10: 
    load_samples = False
    input_steps = 50 #both set to 50 as the default setting for the angle network
    unroll_steps = 50

#percentage used for validation data
test_rate = 0.1 if not load_test else 1

#learning rate for sgd if used
learning_rate = 0.01

print('model: %s (type %s, num %s)' % (model_id,model_type,model_num))
print('%s classes, input_dim %s' % (classes, input_dim))

In [ ]:
#--------- PREPARE DATA --------------
#load samples
if load_samples:
    samples = ls.load(filename)
else:
    samples = ls.Samples(labels)
    
    for label in labels:
        samples.load_samples(label = label, sample_ids = sample_ids)

    samples.convert_to_input(size = None, accel=accel, gyro=gyro, compass=compass)

if add_absolutes:
    samples.add_absolutes()
    input_dim += len(accel)+len(gyro)+len(compass)
if add_moving_average:
    samples.add_moving_average()
    input_dim += len(accel)+len(gyro)+len(compass)
samples.unroll(unroll_steps, overlap_step, complete=complete_unroll)
samples.convert_to_onehot()
samples.split_test(test_rate, random=False)
if not use_rnn: samples.flatten()
    
    
#prepare training data
xtrain,ytrain,seqlen = samples.get_all(padding=False, use_labels_data=use_labels_data)
xtest,ytest,seqlentest = samples.get_test(padding=False, use_labels_data=use_labels_data)
#xtest/ytest is the validation data (or the test data when having the test set loaded)


#normalize data
if normalize:
    
    mean = [0]*input_dim
    maxx = [-100000]*input_dim
    minx = [1000000]*input_dim
    for dataset in [xtrain,xtest]:
        for x in dataset:
            for step in x:
                for d in range(input_dim):
                    mean[d] += step[d]
                    if maxx[d] < step[d]: maxx[d] = step[d]
                    if minx[d] > step[d]: minx[d] = step[d]

    for d in range(input_dim):
        #only for z axis
        #mean[d] = mean[d]/(len(xtrain)*len(xtrain[0])+len(xtest)*len(xtest[0]))
        mean[d] = 0 if d != 2 else 1
        maxx[d] -= mean[d]
        minx[d] -= mean[d]
        
    
    def norm(x):
        for d in range(input_dim):
            x[d] -= mean[d]
            x[d] = (x[d] - minx[d]) / (maxx[d] - minx[d])
            x[d] = x[d]*2 - 1
        
    for dataset in [xtrain,xtest]:
        for x in dataset:
            for step in x:
                norm(step)

    print(mean)
    print(maxx)
    print(minx)
    
    #constants
    mean = [0, 0, 1, 0, 0, 0]
    maxx = [5.1043901443481445, 4.228890419006348, 0.26448965072631836, 0.49148064851760864, 0.2268102616071701, 0.6176088452339172]
    minx = [-2.644476890563965, -18.73173713684082, -0.34867972135543823, -0.6155312657356262, -0.2226802408695221, -0.6029812693595886]



if unroll_steps is None and not predict_mode and not load_test:
    from keras.preprocessing import sequence
    xtrain = sequence.pad_sequences(xtrain)
    xtest = sequence.pad_sequences(xtest)
    #not efficient but whatevs
    if len(xtrain[0]) > len(xtest[0]):
        input_steps = len(xtrain[0])
        xtest = sequence.pad_sequences(xtest, maxlen=input_steps)
    elif len(xtrain[0]) < len(xtest[0]):
        input_steps = len(xtest[0])
        xtrain = sequence.pad_sequences(xtrain, maxlen=input_steps)
    else:
        input_steps = len(xtrain[0])
    
    ytrain = sequence.pad_sequences(ytrain, maxlen=input_steps)
    ytest = sequence.pad_sequences(ytest, maxlen=input_steps)

In [ ]:
#--------- PREPARE MODELS --------------
#create/load model
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout
from keras.layers import LSTM, SimpleRNN
from keras.layers.wrappers import TimeDistributed
from keras.layers.core import Masking
from keras import optimizers
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

if load_rnn and not load_only_weights:
    model = load_model(model_path)
    model.load_weights(weights_path)
else:
    model = Sequential()
    if unroll_steps is None and not predict_mode:
        #variable length -> mask paddings
        #(not implemented for predict_mode)
        model.add(Masking(input_shape=(input_steps, input_dim)))
        
        if use_lstm:
            model.add(LSTM(hidden_layers.pop(0), dropout=dropout_prob, recurrent_dropout=dropout_prob, 
                       return_sequences=use_labels_data))
        else:
            model.add(SimpleRNN(hidden_layers.pop(0), dropout=dropout_prob, recurrent_dropout=dropout_prob, 
                       return_sequences=use_labels_data))
                     
        if use_labels_data:
            for k in hidden_layers:
                model.add(TimeDistributed(Dense(k, activation=activation)))
            model.add(TimeDistributed(Dense(classes, activation='softmax')))
        else:
            for k in hidden_layers:
                model.add(Dense(k, activation=activation))
            model.add(Dense(classes, activation='softmax'))
        
    elif use_rnn:
        #recurrent structure: one recurrent cell followed by time distributed layers
        if use_lstm:
            if predict_mode:
                model.add(LSTM(hidden_layers.pop(0), dropout=dropout_prob, recurrent_dropout=dropout_prob, 
                       batch_input_shape=(1, input_steps, input_dim), return_sequences=use_labels_data, stateful=True))
            else:
                #hidden_layers.append(classes)
                #model.add(LSTM(hidden_layers.pop(0), activation='softmax', dropout=dropout_prob, recurrent_dropout=dropout_prob, 
                #       input_shape=(input_steps, input_dim), return_sequences=use_labels_data))
                model.add(LSTM(hidden_layers.pop(0), dropout=dropout_prob, recurrent_dropout=dropout_prob, 
                       input_shape=(input_steps, input_dim), return_sequences=use_labels_data))
        else:
            if predict_mode:
                model.add(SimpleRNN(hidden_layers.pop(0), dropout=dropout_prob, recurrent_dropout=dropout_prob, 
                       batch_input_shape=(1, input_steps, input_dim), return_sequences=use_labels_data, stateful=True))
            else:
                model.add(SimpleRNN(hidden_layers.pop(0), dropout=dropout_prob, recurrent_dropout=dropout_prob, 
                       input_shape=(input_steps, input_dim), return_sequences=use_labels_data))
                     
        if use_labels_data:
            for k in hidden_layers:
                model.add(TimeDistributed(Dense(k, activation=activation)))
            model.add(TimeDistributed(Dense(classes, activation='softmax')))
        else:
            for k in hidden_layers:
                model.add(Dense(k, activation=activation))
            model.add(Dense(classes, activation='softmax'))
    else:
        #purely DNN
        if not hidden_layers: #(no hidden layers)
            model.add(Dense(classes, activation='softmax', input_shape=(input_steps*input_dim,)))
        else:
            model.add(Dense(hidden_layers.pop(0), activation=activation, input_shape=(input_steps*input_dim,)))
            for k in hidden_layers:
                model.add(Dense(k, activation=activation))
                if dropout_dense: model.add(Dropout(0.2))
            model.add(Dense(classes, activation='softmax'))
   
        
    

    if use_sgd: opt = optimizers.SGD(lr=learning_rate)
    else: opt = 'adam'
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    if predict_mode or load_only_weights:
        #do not save model in predict_mode!
        model.load_weights(weights_path)
    else:
        model.save(model_path)
        model.save_weights(weights_path)
        

print(model.summary())

In [ ]:
#--------- TRAINING/PREDICTION FUNCTIONS --------------
if not predict_mode:
    #only relevant when training, not in predict mode
    """
    Trains the network for the given epochs and batch_size. The network with the best validation accuracy is saved.
    @param epochs: how many epochs to train for
    @param batch_size: the batch size to use
    """
    def train(epochs, batch_size):
        mcp = ModelCheckpoint(weights_path, monitor='val_acc', save_best_only=True, save_weights_only=True)
        model.fit(xtrain, ytrain,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(xtest, ytest),
                  callbacks=[mcp])
        score, acc = model.evaluate(xtest, ytest, batch_size=len(xtest))
        print('Test score: %s' % score)
        print('Test accuracy: %s' % acc)
        #model.save(model_path)
        #model.save_weights(weights_path)
        
"""
Test all available data (both train and validation set).
@param batch_size: feed batches of this size to reduce memory needed
"""    
def test_total(batch_size=None):
    if batch_size == None:
        acc1 = model.evaluate(xtrain, ytrain, batch_size=len(xtrain), verbose=0)[1]
        acc2 = model.evaluate(xtest, ytest, batch_size=len(xtest), verbose=0)[1]
    else:
        acc1 = 0
        num = len(xtrain)
        for i in range(0,num,batch_size):
            size = min(batch_size, len(xtrain)-i)
            xtrain2 = xtrain[i:i+size]
            ytrain2 = ytrain[i:i+size]
            acc1 += (float(size)/num) * model.evaluate(xtrain2, ytrain2, batch_size=size, verbose=0)[1]

        acc2 = 0
        num = len(xtest)
        for i in range(0,num,batch_size):
            size = min(batch_size, len(xtest)-i)
            xtrain2 = xtest[i:i+size]
            ytrain2 = ytest[i:i+size]
            acc2 += (float(size)/num) * model.evaluate(xtrain2, ytrain2, batch_size=size, verbose=0)[1]
    acc = acc1*(1.0-test_rate) + acc2*test_rate
    print('Train accuracy: %s' % acc1)
    print('Test accuracy: %s' % acc2)
    print('Total accuracy: %s' % acc)

"""
Evaluate the model using the test/validation data.
"""
def test():
    score, acc = model.evaluate(xtest, ytest, batch_size=len(xtest))
    print('Test score: %s' % score)
    print('Test accuracy: %s' % acc)


"""
Calculates the per-label accuracies.
@param test: if true, the test/validation set is used
"""
def per_label_acc(test=True):
    if test:
        x,y = xtest,ytest
    else:
        x,y = xtrain,ytrain
    accs = [0] * classes
    counts = [0] * classes

    print_step = len(y)/10
    for i in range(len(y)):
        acc = model.evaluate(np.array([x[i]]), np.array([y[i]]), batch_size=1, verbose=0)[1]
        c = 0
        if use_labels_data:
            for y_oh in y[i]:
                c = np.argmax(y_oh)
                accs[c] += acc
                counts[c] += 1
        else:
            c = np.argmax(y[i])
            accs[c] += acc
            counts[c] += 1
        if i%print_step==0:
            print('Progress: %s' % (float(i)/len(y)))

    acc_total = 0
    for i in range(classes):
        acc = accs[i]/counts[i]
        print('%sx %s: %s' % (counts[i],labels[i],acc))
        acc_total += acc*1.0/classes
    print('Total test acc: %s' % acc_total)

"""
Interface to make feeding data through a recurrent network simpler.
@param s: list of data points, if feeding only a single timestep, this needs to be a list with one element
@param as_prob: return the output as probabilities instead of argmax
@return a list with the output of the network for each of the input steps
"""
def feed_seq(s, as_prob=False):
    if normalize:
        for step in s:
            norm(step)
    res = model.predict(np.array([s]),batch_size=1)[0]
    resarg = []
    for r in res:
        if as_prob: resarg.append(r)
        else: resarg.append(np.argmax(r))
    return resarg


In [ ]:
#small test: compare output of network for one example sample (for each label)
if predict_mode:
    if False:
        import time
        start = time.time()
        for i in range(10000):
            z = [0,0,0,0,0,0]
            v = [z]*1
            feed_seq(v)
        dt = time.time() - start
        print('%s hz' % (1/(dt/10000)))

    if True:
        for key in samples.test:
            model.reset_states()
            xdata = samples.test[key][0][0]
            ydata = samples.labels_test[key][0][0]

            res = feed_seq(xdata)

            res_predicted = res
            res_expected = []
            acc = 0
            for i in range(len(res)):
                res_expected.append(np.argmax(ydata[i]))
                if res_predicted[len(res_expected)-1] == res_expected[-1]:
                    acc += 1.0
            print('%s: %s' % (key, acc/len(res)))
            print(res_predicted)
            print(res_expected)

In [ ]:
#display how many samples are available
if True:
    for key in labels:
        print('%s: %s / %s' % (key, len(samples.data[key]), len(samples.test[key])))

In [ ]:
#train the network
if train_network and not predict_mode:
    #train(1000,32)
    
    #model.optimizer.lr.assign(0.00000000000001)
    train(2000,40)
    
    #per_label_acc()

In [ ]:
#evaluate model
if True:
    #if not load_test: test_total(1000)
    #elif unroll_steps is not None: test()
    #test()
    per_label_acc()

In [ ]:
#live demo
%env ROS_HOSTNAME=zinunb
%env ROS_IP=$(hostname -I)
%env ROS_MASTER_URI=http://pi:11311

if live_demo and predict_mode:
    import imu_listener as il
    #reset()
    il.imu_listener(feed_seq,labels,1,5)
    print('Exited')

In [ ]:
"""
Returns the two most likely labels from the given data (like argmax, but the top two are returned).
"""
def argmax2(data):
        first = np.argmax(data)
        p = data[first]
        data[first] = -1 #probabilities are >0
        second = np.argmax(data)
        data[first] = p
        return first,second

In [ ]:
#test if 30deg network recognizes 15deg
if model_type == 11 and use_rnn and True and not predict_mode:
    def get_expected(key):
        label1,label2 = '',''
        
        import re
        import math
        
        vals = re.match(r'angle_(\d*)(?:_(l|r))?', key).groups()
        angle = int(vals[0])
        left = vals[1]
        
        angle1 = angle-15
        angle2 = angle+15
        
        if angle1 == 0:
            label1 = 'angle_0'
        else:
            label1 = 'angle_%s_%s' % (angle1,left)
            
        if angle2 == 180:
            label2 = 'angle_180'
        else:
            label2 = 'angle_%s_%s' % (angle2,left)

        return labels.index(label1),labels.index(label2)
    
    labels2,sample_ids2,filename2 = model_types.get_labels(12)
    samples2 = ls.Samples(labels2)

    for label in labels2:
        samples2.load_samples(label = label, sample_ids = sample_ids2)

    samples2.convert_to_input(size = None, accel=accel, gyro=gyro, compass=compass)

    if add_absolutes:
        samples2.add_absolutes()
    if add_moving_average:
        samples2.add_moving_average()
    samples2.unroll(unroll_steps, overlap_step)
    samples2.convert_to_onehot()
    
    #test on all
    for key in samples2.data:
        if key in labels: continue
        acc = 0
        correct = 0
        count = len(samples2.data[key])*2
        prob_avg = 0
        prob_min = 2
        prob_max = -1
        prob_min_avg = 0
        prob_max_avg = 0
        prob_1_avg = 0
        prob_2_avg = 0
        
        expected1,expected2 = get_expected(key)
        print('%s: %s and %s expected' % (key,labels[expected1],labels[expected2]))
        for i in range(len(samples2.data[key])):
            xdata = samples2.data[key][i][0]
            ydata = samples2.labels_data[key][i][0]

            res = model.predict(np.array([xdata]))[0]
            r = argmax2(res)
            
            if expected1 in r: acc += 1
            if expected2 in r: acc += 1
            if expected1 in r and expected2 in r:
                correct += 1
                
                p1 = res[r[0]]
                p2 = res[r[1]]
                
                prob_min_avg += p2 if p1>p2 else p1
                prob_max_avg += p1 if p1>p2 else p2
                
                if r[0] == expected1:
                    prob_1_avg += p1
                    prob_2_avg += p2
                else:
                    prob_1_avg += p2
                    prob_2_avg += p1
                
                for p in [p1,p2]:
                    prob_avg += p
                    if p < prob_min: prob_min = p
                    if p > prob_max: prob_max = p
                
                
        print('Accuracy %s: %s (%s of %s)' % (key,acc/(1.0*count),acc,count))
        count = count/2
        print('Correct %s: %s (%s of %s)' % (key,correct/(1.0*count),correct,count))
        print('Probs (min/max/min_avg/max_avg): %s, %s, %s, %s' % (prob_min, prob_max, prob_min_avg/count, prob_max_avg/count))
        print('Probs (avg/avg %s/avg %s): %s, %s, %s' % (labels[expected1],labels[expected2],prob_avg/count,prob_1_avg/count,prob_2_avg/count))
        print('====================================================================================================')
    
        
    #test for human eyes
    for key in samples2.data:
        if key in labels: continue
        xdata = samples2.data[key][0][0]
        ydata = samples2.labels_data[key][0][0]

        res = model.predict(np.array([xdata]))[0]
        r1,r2 = argmax2(res)
        
        print("%s: %s and %s (%s and %s)" % (key,labels[r1],labels[r2],'{0:.0f}%'.format(res[r1]*100),'{0:.0f}%'.format(res[r2]*100)))
    

In [ ]:
#test if curve network recognizes sm/ml curves
if model_type == 0 and not predict_mode and True:
    def get_expected(key):
        label1,label2 = '',''
        
        left = '_l' if key.endswith('l') else '_r'
        
        label1 = 'curve_m'
        if key.startswith('curve_sm'):
            label2 = 'curve_s'
        elif key.startswith('curve_ml'):
            label2 = 'curve_l'
        
        label1 += left
        label2 += left
        return labels.index(label1),labels.index(label2)
    
    labels2,sample_ids2,filename2 = model_types.get_labels(6)
    print(labels2)
    samples2 = ls.load(filename2)
    samples2.unroll(None, 1)
    samples2.convert_to_onehot()
    
    #test on all
    for key in samples2.data:
        if key in labels: continue
        acc = 0
        correct = 0
        count = 0
        prob_avg = 0
        prob_min = 2
        prob_max = -1
        prob_min_avg = 0
        prob_max_avg = 0
        prob_1_avg = 0
        prob_2_avg = 0
        count_exp1 = 0
        count_exp2 = 0
        labeldict = {}
        for i in range(len(labels)):
            labeldict[i] = 0
        
        expected1,expected2 = get_expected(key)
        print('%s: %s and %s expected' % (key,labels[expected1],labels[expected2]))
        for i in range(len(samples2.data[key])):
            xdata = samples2.data[key][i][0]
            ydata = samples2.labels_data[key][i][0]
            
            model.reset_states()
            res_all = model.predict(np.array([xdata]))[0]
            for i in range(len(ydata)):
                if np.argmax(ydata[i]) != 0:
                    res = res_all[i]
                    count += 2
                    r = argmax2(res)
                    
                    for k in r:
                        labeldict[k] += 1

                    if expected1 in r: 
                        acc += 1
                        count_exp1 += 1
                    if expected2 in r: 
                        acc += 1
                        count_exp2 += 1
                    if expected1 in r and expected2 in r:
                        correct += 1

                        p1 = res[r[0]]
                        p2 = res[r[1]]

                        prob_min_avg += p2 if p1>p2 else p1
                        prob_max_avg += p1 if p1>p2 else p2

                        if r[0] == expected1:
                            prob_1_avg += p1
                            prob_2_avg += p2
                        else:
                            prob_1_avg += p2
                            prob_2_avg += p1

                        for p in [p1,p2]:
                            prob_avg += p
                            if p < prob_min: prob_min = p
                            if p > prob_max: prob_max = p
                
                
        print('Accuracy %s: %s (%s of %s)' % (key,acc/(1.0*count),acc,count))
        count = count/2
        print('Correct %s: %s (%s of %s)' % (key,correct/(1.0*count),correct,count))
        print('Counts %s: %s (%s of %s)' % (labels[expected1],count_exp1/(1.0*count),count_exp1,count))
        print('Counts %s: %s (%s of %s)' % (labels[expected2],count_exp2/(1.0*count),count_exp2,count))
        print('Probs (min/max/min_avg/max_avg): %s, %s, %s, %s' % (prob_min, prob_max, prob_min_avg/count, prob_max_avg/count))
        print('Probs (avg/avg %s/avg %s): %s, %s, %s' % (labels[expected1],labels[expected2],prob_avg/count,prob_1_avg/count,prob_2_avg/count))
        for a in labeldict:
            print('%s: %s (%s of %s)' % (labels[a],labeldict[a]/(1.0*count),labeldict[a],count))
        print('====================================================================================================')

In [ ]:
if predict_mode and True:
    #%matplotlib notebook
    import matplotlib
    import matplotlib.pyplot as plt
    DIM = input_dim
    fig = plt.figure()
    ax = fig.add_subplot(111)
    #ax = fig.add_axes([0.1,0.1,0.75,0.75])
    plt.ion()
    #fig.show()
    fig.canvas.draw()

    def draw_data_cutter(key, expected, omit=[], index=0, compare=False):
        model.reset_states()
        xdata = samples.test[key][index][0]
        ydata = samples.labels_test[key][index][0]

        res = feed_seq(xdata)

        res_predicted = res
        res_expected = []
        for i in range(len(res)):
            res_expected.append(np.argmax(ydata[i]))
        
        ax.clear()
        ax.set_title('%s (%s)' % (key, 'expected' if expected else 'predicted'))
        ax.set_xlabel('time')

        data2 = np.array(xdata)
        data = []
        for i in range(DIM): data.append([])
        for d in data2:
            for i in range(DIM):
                if i in omit: continue
                data[i].append(d[i])

        for i in range(DIM):
            if i in omit: continue
            if i<3 and DIM>3:
                label = 'accel %s' % ('xyz'[i])
            else:
                label = 'gyro %s' % ('xyz'[i-3])
            ax.plot(range(len(data[i])),data[i], label=label)
        
        if compare:
            if expected:
                span_data = res_expected
                key_expected = labels.index(key)
            else:
                key_expected = labels.index(key)
                span_data = res_predicted
                for i in range(len(span_data)):
                    if span_data[i] != res_expected[i]:
                        span_data[i] = -1 #different
                    elif res_expected[i] == 0 and key_expected != 0:
                        span_data[i] = 0 #same, but straight in a non straight sample
                    else:
                        span_data[i] = 1 #same

                key_expected = 1 
        else:
            span_data = res_expected if expected else res_predicted
            key_expected = labels.index(key)
            
            
        start_i = 0
        current_type = 0
        for i in range(len(span_data)):
            if current_type != span_data[i]:
                if current_type == key_expected:
                    ax.axvspan(start_i, i, color='green', alpha=0.1)
                elif current_type != key_expected and current_type != 0:
                    ax.axvspan(start_i, i, color='red', alpha=0.1)
                current_type = span_data[i]
                start_i = i

        if current_type == key_expected:
            ax.axvspan(start_i, len(span_data), color='green', alpha=0.1)
        elif current_type != key_expected and current_type != 0:
            ax.axvspan(start_i, len(span_data), color='red', alpha=0.1)

        
        ax.legend(loc='lower left')
        fig.canvas.draw()
        

In [ ]:
#save image of the predicition for a single sample per label
if predict_mode and True:
    omit = [0,1,2] if model_type==0 else []
    for label in labels:
        draw_data_cutter(label,True,omit)
        plt.savefig('data-%s-%s.png' % (label, 'expected'),pad_inches=0,bbox_inches='tight')
        draw_data_cutter(label,False,omit)
        plt.savefig('data-%s-%s.png' % (label, 'predicted'),pad_inches=0,bbox_inches='tight')
        print('Saved %s' % label)
    print("Done!")

In [ ]:
#save image of the prediction for all test samples
if predict_mode and load_test and False:
    omit = [0,1,2] if model_type==0 else []
    for label in labels:
        for index in range(20):
            draw_data_cutter(label,True,omit,index)
            plt.savefig('test/data-%s-%s-%s.png' % (label, index, 'expected'),pad_inches=0,bbox_inches='tight')
            draw_data_cutter(label,False,omit,index)
            plt.savefig('test/data-%s-%s-%s.png' % (label, index, 'predicted'),pad_inches=0,bbox_inches='tight')
            print('Saved %s' % label)
    print("Done!")